In [ ]:
import os

from veil.veil_client import *
from veil.constants import NetworkId, NETWORK_INFO
from utils.logutils import setup_logger
import logging

LOGGER = setup_logger("VeilPlayground")
setup_logger("veil.veil_client", log_level=logging.DEBUG)

NETWORK_ID = NetworkId.KOVAN
PRIVATE_KEY = os.environ.get("PRIVATE_KEY")
INFURA_API_KEY = os.environ.get("INFURA_API_KEY")
WEB3_RPC_URL = "https://{}.infura.io/{}".format(NETWORK_ID.name.lower(), INFURA_API_KEY)

client = VeilClient(
    network_id=NETWORK_ID.value,
    web3_rpc_url=WEB3_RPC_URL,
    private_key=PRIVATE_KEY,
)

MY_ADDRESS = client.account_address

print(
    "NETWORK:",  NETWORK_ID,
    "\nRPC_URL", WEB3_RPC_URL,
    "\nADDRESS IN USE:", MY_ADDRESS,
    "\nETH BALANCE:", client.get_eth_balance()
)

print(client)
print("Signing into veil...")
print(client._authenticate())

In [ ]:
client.get_eth_balance()

In [ ]:
markets = client.get_markets(
#     channel="btc", 
    status="open",
#     page=0
#     raw_json=True,
)
print(len(markets))
markets

In [ ]:
from pprint import pprint
print(set([m.type for m in markets]))
print(set([m.channel for m in markets]))
print(set([m.status for m in markets]))
print([m.metadata for m in markets])
print([(m.type.value, m.num_ticks) for m in markets])

In [ ]:
market = client.get_market(
    "bnb-usd-2019-04-12-5x",
#     raw_json=True
)
market

In [ ]:
min_max = [x / TEN_18 for x in  [market.min_price, market.max_price]]
price_range = (min_max[1] - min_max[0])
min_max, price_range, (15.08 - min_max[0]) / price_range

In [ ]:
from veil.veil_client import TokenType, OrderSide

bids_long = client.get_bids(
    market_slug=market.slug,
    token_type=TokenType.LONG,
)
bids_short = client.get_bids(
    market_slug=market.slug,
    token_type=TokenType.SHORT,
)
asks_long = client.get_asks(
    market_slug=market.slug,
    token_type=TokenType.LONG,
)
asks_short = client.get_asks(
    market_slug=market.slug,
    token_type=TokenType.SHORT,
)
bids_long, bids_short, asks_long, asks_short

In [ ]:
order_fills = client.get_order_fills(
    market_slug=market.slug,
    token_type=TokenType.LONG,
#     raw_json=True
)
order_fills

In [ ]:
balances = client.get_balances(market.slug)
balances

In [ ]:
order_res = client.post_order(
    market=market,
    token_type="short",
    side="buy",
    amount=0.005,
    price=0.4,
)
order_res

In [ ]:
from veil.veil_client import OrderStatus

my_orders = client.get_orders(
    market.slug,
#     order_status=None
#     raw_json=True
)
my_orders

In [ ]:
for order in my_orders:
    if order.status == OrderStatus.OPEN:
        cancel_res = client.cancel_order(order.uid)
        print(cancel_res)

In [ ]:
feed_data = client.get_feed_data(market.index)
print(feed_data)
feed_data.entries.head()